In [ ]:
!pip install boto3

In [3]:
import boto3
import time
from google.colab import userdata

In [10]:
# Crie uma sessao usando as credenciais do usuario IAM
session = boto3.Session(
    aws_access_key_id = userdata.get('aws_access_key_id'),
    aws_secret_access_key = userdata.get('aws_secret_access_key'),
    region_name = 'us-east-2'
)

In [11]:
# Crie um cliente SQS
service_sqs = session.client('sqs')
sqs_queue_name = 'eda-sqs-msg.fifo'

In [12]:
# Obtenha a URL da fila
response = service_sqs.get_queue_url(QueueName = sqs_queue_name)
queue_url = response['QueueUrl']  # URL da fila criada

In [ ]:
# Loop para receber mensagens da fila
while True:
    # Recebe mensagens da fila (maximo de 10 mensagens por vez)
    response = service_sqs.receive_message(
        QueueUrl=queue_url,
        MaxNumberOfMessages=10, # Numero maximo de mensagens a serem recebidas
        MessageAttributeNames=[
            'All'
        ],
        VisibilityTimeout=30,
        WaitTimeSeconds=20 # Tempo maximo de espera por mensagens (long polling)
    )

    # Verifica se ha mensagens na resposta
    if 'Messages' in response:
        for message in response['Messages']:
            # Imprime o corpo da mensagem
            print("Mensagem recebida:", message)

            # Exclui a mensagem da fila apos processa-la
            service_sqs.delete_message(
                QueueUrl=queue_url,
                ReceiptHandle=message['ReceiptHandle']
            )
    else:
        print("Sem mensagens na fila. Aguardando...")

    time.sleep(5)  # Aguarda 5 segundos antes de verificar novamente